In [2]:
from spotipy.oauth2 import SpotifyClientCredentials
import spotipy
import billboard
import string
#import applemusicpy
import lyricsgenius
#import SpotifyCharts as sCharts
import csv
import os
from textstat.textstat import textstat
import threading as thread
import re
from nltk.sentiment.vader import SentimentIntensityAnalyzer
import numpy as np
from sklearn import linear_model, mixture
from sklearn.metrics import mean_squared_error, r2_score, accuracy_score
from sklearn.linear_model import LogisticRegression
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import GaussianNB
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.ensemble import RandomForestClassifier
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from wordcloud import WordCloud
import matplotlib.pyplot as plt
import nltk
from collections import Counter
from keras.preprocessing.text import Tokenizer
from keras.models import Sequential
from keras.layers import LSTM, Dense, Dropout, Masking, Embedding

Using TensorFlow backend.


In [3]:
path = "C:/Users/Chaitu Konjeti/SongPopularityPredictionAlgorithm/output"
genius = lyricsgenius.Genius("Xf0XfBJTZon0Sra2rGV56TAXp6jOUaLJVhmHxqbTW5mp-j6S2NVcmHWSLQ29v0dk")

In [4]:
try:
    os.mkdir(path)
except:
    pass

In [5]:
def writeSongCharacteristics(i, billboardChart, songs):
    key = str(billboardChart[i].title + billboardChart[i].artist)

    if key not in songs:
        song = billboardChart[i]
        title = billboardChart[i].title
        artist = song.artist
        peakpos = song.peakPos
        lastpos = song.lastPos
        numWeeks = song.weeks
        currentPos = song.rank
        isNew = song.isNew
        lyrics = genius.search_song(title, artist).lyrics
        row = [title, artist, lyrics, peakpos, lastpos, numWeeks, currentPos, isNew]
        songs[key] = row

    if key in songs:
        if billboardChart[i].peakPos != songs.get(key)[3]:
            newRow = songs.get(key)
            newRow[3] = billboardChart[i].peakPos
            songs[key] = newRow
        if billboardChart[i].weeks != songs.get(key)[5]:
            newRow = songs.get(key)
            newRow[5] = billboardChart[i].weeks
            songs[key] = newRow

In [6]:
def getAllSongData(startMonth, startYear, endMonth, endYear, numSongs):
    for year in range(startYear, endYear + 1):
        thread.Thread(None, target=yearlySongData, args=(year, startMonth, endMonth, numSongs)).start()

In [7]:
def yearlySongData(year, startMonth, endMonth, numSongs):
    for month in range (startMonth, endMonth + 1):
            thread.Thread(None, target=monthlySongData, args=(year, month, numSongs)).start()

In [8]:
def monthlySongData(year, month, numSongs):
        outputFileName = "C:/Users/Chaitu Konjeti/SongPopularityPredictionAlgorithm/output/billboardHot100_Lyrics_{}_{}.csv".format(year, month)

        with open(outputFileName, 'a+', newline='', encoding='utf-8') as outputFile:
            songs = {}
            day = 1
            dataWriter = csv.writer(outputFile)

            while (day <= 31):
                if month in range(1, 10):
                    try:
                        billboardChart = billboard.ChartData('hot-100', date = "{}-{:02d}-{}".format(year, month, day))
                        for i in range(0, numSongs):
                            try:
                                writeSongCharacteristics(i, billboardChart, songs)
                            except:
                                pass
                    except:
                        pass

                elif month in range(10, 13):
                    try:
                        billboardChart = billboard.ChartData('hot-100', date = "{}-{}-{}".format(year, month, day))
                        for i in range(0, numSongs):
                            try:
                                writeSongCharacteristics(i, billboardChart, songs)
                            except:
                                pass
                    except:
                        pass
                day += 5

            for key in songs:
                if (len(songs[key][2]) < 10000):
                    dataWriter.writerow(songs[key])

In [9]:
def lyric_preprocessing(dir):
    for filename in os.listdir(dir):
        data = []
        sid = SentimentIntensityAnalyzer()
        with open(dir + filename, encoding='utf-8', mode='r+') as csv_file:
            csv_reader = csv.reader(csv_file, delimiter=',')
            for row in csv_reader:
                temp = row
                num_positive = 0
                num_negative = 0
                num_neutral = 0
                dale_chall_score = 0
                #print(row[0])
                lyrics = row[2]
                lyrics = re.sub(r'[\(\[].*?[\)\]]', '', lyrics)
                lyrics = os.linesep.join([s for s in lyrics.splitlines() if s])
                #print(lyrics)
                #this_sentence = lyrics.decode('utf-8')
                #numlines = 0
                for line in lyrics.splitlines():
                    #print(line)
                    comp = sid.polarity_scores(line)
                    #print(comp)
                    comp = comp['compound']
                    if comp >= 0.5:
                        num_positive += 1
                    elif comp > -0.5 and comp < 0.5:
                        num_neutral += 1
                    else:
                        num_negative += 1

                    #numlines += 1

                temp.append(num_positive)
                temp.append(num_neutral)
                temp.append(num_negative)
                temp.append(textstat.dale_chall_readability_score(lyrics))
                temp.append(textstat.difficult_words(lyrics))
                temp.append(lyrics)
                data.append(temp)
                #print(temp)
            outputFileName = "C:/Users/Chaitu Konjeti/song-popularity-prediction-algorithm/processed/" + filename

            with open(outputFileName, 'a+', newline='', encoding='utf-8') as outputFile:
                dataWriter = csv.writer(outputFile)
                for items in data:
                    #print(items)
                    dataWriter.writerow(items)

In [ ]:
getAllSongData(1, 2000, 12, 2019, 100)

In [30]:
lyric_preprocessing("output/")

KeyboardInterrupt: 

In [4]:
X = []
y = []

In [5]:
dir = 'C:/Users/Chaitu Konjeti/song-popularity-prediction-algorithm/processed/'

In [6]:
for filename in os.listdir(dir):
    with open(dir + filename, encoding='utf-8-sig', mode='r') as csv_file:
        df = pd.read_csv(dir + filename, header = None)
        #print(df.head())
        df.columns = ['title', 'artist', 'original', 'peakpos', 'lastpos', 'numWeeks', 'currentPos', 'isNew', 'pos', 'neu', 'neg', 'dale', 'diff', 'lyrics']
        for lyric in df['lyrics']:
            X.append(lyric)
        for pos in df['currentPos']:
            y.append(pos)

In [7]:
len(X)

1483

In [8]:
tokens = []
for post in X:
    tokens += word_tokenize(post)
stop_words = ["I", "i", "me", "my", "myself", "we", "our", "ours", "ourselves", "you", "your", "yours", "yourself", "yourselves", "he", "him", "his", "himself", "she", "her", "hers", "herself", "it", "its", "itself", "they", "them", "their", "theirs", "themselves", "what", "which", "who", "whom", "this", "that", "these", "those", "am", "is", "are", "was", "were", "be", "been", "being", "have", "has", "had", "having", "do", "does", "did", "doing", "a", "an", "the", "and", "but", "if", "or", "because", "as", "until", "while", "of", "at", "by", "for", "with", "about", "against", "between", "into", "through", "during", "before", "after", "above", "below", "to", "from", "up", "down", "in", "out", "on", "off", "over", "under", "again", "further", "then", "once", "here", "there", "when", "where", "why", "how", "all", "any", "both", "each", "few", "more", "most", "other", "some", "such", "no", "nor", "not", "only", "own", "same", "so", "than", "too", "very", "s", "t", "can", "will", "just", "don", "should", "now"]
tokens = [w for w in tokens if not w in stop_words]

tokens = [word for word in tokens if word.isalpha()]
# print(tokens)

In [9]:
vocabulary = set(tokens)
# print(vocabulary)
print(len(vocabulary))

13716


In [10]:
frequency_dist = nltk.FreqDist(vocabulary)

In [11]:
tokenizer = Tokenizer(num_words=None, 
                      filters='!"#$%&()*+,-./:;<=>?@[\\]^_~\t\n',
                      lower = True, split = ' ')

tokenizer.fit_on_texts(X)

sequences = tokenizer.texts_to_sequences(X)
sequences = np.asarray(sequences)
sequences.shape

(1483,)

In [12]:
features = []
labels = []
for i in range(len(sequences)):
    label = y[i]
    #print(label)
    for j in range(0, len(sequences[i]), 50):
        len_of_50 = sequences[i][j:j + 50]
        if len(len_of_50) == 50:
            features.append(sequences[i][j:j + 50])
            labels.append(label)
features = np.asarray(features)
labels = np.asarray(labels)

In [13]:
print(features.shape)
print(labels.shape)

(12264, 50)
(12264,)


In [14]:
features

array([[ 382,    3,   31, ...,   75,   87,   26],
       [   2,    7,   98, ...,   69,  109,   90],
       [   3,   20,  452, ...,   51,   48,   11],
       ...,
       [ 263, 1596, 2632, ...,    3, 5651,   30],
       [  14,  112,   27, ...,    3, 2156,  589],
       [  49,   49,  589, ..., 1023,    1,  618]])

In [15]:
count_vectorizer = CountVectorizer(analyzer='word', ngram_range=(1, 3))
X = count_vectorizer.fit_transform(X)
y = np.asarray(y)

In [49]:
def smape(A, F):
    return 100/len(A) * np.sum(2 * np.abs(F - A) / (np.abs(A) + np.abs(F)))

In [16]:
X_train, X_test, y_train, y_test = train_test_split(features, labels, test_size=0.2, random_state=1)
print(X_train)

[[  18   43    9 ...   40  746    9]
 [  24 4609   28 ...  117  159  105]
 [   2 1299 1871 ...  890   57    2]
 ...
 [  34  689  140 ... 3031 2683 1010]
 [ 168    3  214 ... 1963   45   83]
 [  67   45   10 ... 4237   13   16]]


In [65]:
model = Sequential()

# Embedding layer
model.add(
    Embedding(
        input_dim=20000,
        output_dim=1000,
        weights=None,
        trainable=True))

# Recurrent layer"
model.add(
    LSTM(
        600, return_sequences=False, dropout=0.1,
        recurrent_dropout=0.1))

# Fully connected layer
model.add(Dense(300, activation='relu'))

# Dropout for regularization
model.add(Dropout(0.5))

# Output layer
model.add(Dense(101, activation='softmax'))

# Compile the model
model.compile(
    optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

model.summary()

Model: "sequential_10"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_10 (Embedding)     (None, None, 1000)        20000000  
_________________________________________________________________
lstm_10 (LSTM)               (None, 600)               3842400   
_________________________________________________________________
dense_19 (Dense)             (None, 300)               180300    
_________________________________________________________________
dropout_10 (Dropout)         (None, 300)               0         
_________________________________________________________________
dense_20 (Dense)             (None, 101)               30401     
Total params: 24,053,101
Trainable params: 24,053,101
Non-trainable params: 0
_________________________________________________________________


In [66]:
model.fit(X_train,y_train, 
        batch_size=1500, epochs=30)

Epoch 1/30
9811/9811 [==============================] - 134s 14ms/step - loss: 4.6115 - accuracy: 0.0158
Epoch 2/30
9811/9811 [==============================] - 128s 13ms/step - loss: 4.5820 - accuracy: 0.0192
Epoch 3/30
9811/9811 [==============================] - 129s 13ms/step - loss: 4.5797 - accuracy: 0.0361
Epoch 4/30
9811/9811 [==============================] - 130s 13ms/step - loss: 4.5134 - accuracy: 0.0387
Epoch 5/30
9811/9811 [==============================] - 128s 13ms/step - loss: 4.4122 - accuracy: 0.0434
Epoch 6/30
9811/9811 [==============================] - 130s 13ms/step - loss: 4.3384 - accuracy: 0.0614
Epoch 7/30
9811/9811 [==============================] - 130s 13ms/step - loss: 4.1741 - accuracy: 0.0761
Epoch 8/30
9811/9811 [==============================] - 130s 13ms/step - loss: 3.9582 - accuracy: 0.0985
Epoch 9/30
9811/9811 [==============================] - 132s 13ms/step - loss: 3.7429 - accuracy: 0.1300
Epoch 10/30
9811/9811 [==============================] 

In [67]:
nn_pred = model.predict(X_test)
nn_pred = np.argmax(nn_pred, axis=1)
smape(nn_pred, y_test)

41.0745322915169

In [19]:
y_test

array([15, 84, 44, ..., 28, 99, 47])

In [20]:
clf = GaussianNB()
clf.fit(X_train, y_train)

GaussianNB(priors=None, var_smoothing=1e-09)

In [21]:
clf.score(X_train, y_train)

0.042605239017429415

In [22]:
pred = clf.predict(X_test)

In [23]:
pred

array([18,  3, 11, ..., 62, 57, 47])

In [24]:
y_test

array([15, 84, 44, ..., 28, 99, 47])

In [25]:
def smape(A, F):
    return 100/len(A) * np.sum(2 * np.abs(F - A) / (np.abs(A) + np.abs(F)))

In [26]:
smape(y_test, pred)

92.41495519101447

In [28]:
accuracy_score(y_test, pred)

0.019160211985324093

In [33]:
rff_clf = RandomForestClassifier(max_depth=30)
rff_clf.fit(X_train, y_train)
rff_clf.score(X_train, y_train)

C:\Users\Chaitu Konjeti\Anaconda3\lib\site-packages\sklearn\ensemble\forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


0.4517378452757109

In [34]:
pred = (rff_clf.predict(X_test))
pred

array([13, 86,  3, ..., 47, 98, 47])

In [35]:
smape(y_test, pred)

46.237533765059645

In [36]:
accuracy_score(y_test, pred)

0.09539339584182634

In [89]:
logr_clf = LogisticRegression()
logr_clf.fit(X_train, y_train)
logr_clf.score(X_train, y_train)

C:\Users\Chaitu Konjeti\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\Users\Chaitu Konjeti\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:469: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)
C:\Users\Chaitu Konjeti\Anaconda3\lib\site-packages\sklearn\svm\base.py:929: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


0.05126898379370095

In [37]:
pred = logr_clf.predict(X_test)
pred

NameError: name 'logr_clf' is not defined

In [38]:
smape(y_test, pred)

46.237533765059645